In [1]:
# Import the required libraries
import pandas as pd
import numpy as np

In [2]:
# Load the data 
df = pd.read_csv('Iris.csv')
df=df.drop(columns=['Id'],axis=1)
df

SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm         Species
0              5.1           3.5            1.4           0.2     Iris-setosa
1              4.9           3.0            1.4           0.2     Iris-setosa
2              4.7           3.2            1.3           0.2     Iris-setosa
3              4.6           3.1            1.5           0.2     Iris-setosa
4              5.0           3.6            1.4           0.2     Iris-setosa
..             ...           ...            ...           ...             ...
145            6.7           3.0            5.2           2.3  Iris-virginica
146            6.3           2.5            5.0           1.9  Iris-virginica
147            6.5           3.0            5.2           2.0  Iris-virginica
148            6.2           3.4            5.4           2.3  Iris-virginica
149            5.9           3.0            5.1           1.8  Iris-virginica

[150 rows x 5 columns]

In [3]:
def calculate_gini_index(dataset,feature,label):
    dictionary = {}
    i = 0
    for values in dataset[feature]:
#         print(values)
        dictionary[values] = []
    if not dictionary:
        return 0
    for values in dataset[feature]:
#         print(values)
        try:
            dictionary[values].append(dataset[label][i])
        except:
            return 1
        i+=1
#     print(dictionary)
    gini = 0
    for value in dictionary.values():
        temp = np.array(value)
        unq,class_counts = np.unique(temp,return_counts=True)
        p = class_counts/np.sum(class_counts)
        gini_c = 1 - np.sum(p**2)
        gini += np.sum(class_counts)/len(dataset[label])*gini_c
    return gini

In [4]:
# Set the features and label
features = df.columns[:-1]
label = 'Species'
parent=None
features

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object')

In [5]:
#Define the create decision tree function
def create_decision_tree(dataset, df, features, label, parent):
    datum = np.unique(df[label],return_counts=True)
    unique_data = np.unique(dataset[label])
    if len(unique_data) == 1:
        return unique_data[0]
    elif len(dataset) == 0:
        return unique_data[np.argmax(datum[1])]
    elif len(features) == 0:
        return parent
    else:
        parent = unique_data[np.argmax(datum[1])]
        # Call the calculate_gini_index function
        item_values = [calculate_gini_index(dataset, feature, label) for feature in features]
        optimum_feature_index = np.argmax(item_values)
        optimum_feature = features[optimum_feature_index]
        decision_tree = {optimum_feature:{}}
        features = [i for i in features if i != optimum_feature] 
        for value in np.unique (dataset [optimum_feature]):
            min_data = dataset. where(dataset[optimum_feature] == value).dropna()
            # Recusrsive call to create_decision_tree function
            min_tree = create_decision_tree(min_data, df,features, label, parent)
            decision_tree[optimum_feature][value] = min_tree
        return(decision_tree)

In [6]:
# Train the decision tree model
decision_tree = create_decision_tree(df,df,features,label,parent)

In [7]:
def predict(test_data,decision_tree):
    for nodes in decision_tree.keys():
        value = test_data[nodes]
        decision_tree = decision_tree[nodes][value]
        
        prediction = 0
        if type(decision_tree) is dict:
            prediction = predict(test_data,decision_tree)
        else:
            prediction = decision_tree
            break
    return prediction

In [8]:
# Predict using the trained model
sample_data = {'SepalLengthCm':5.9,'SepalWidthCm':3.0,'PetalLengthCm':5.1,'PetalWidthCm':1.8}
test_data = pd.Series(sample_data)
prediction = predict(test_data,decision_tree)
prediction

'Iris-virginica'